# Font Diffusion Notebook

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
from PIL import ImageFont, ImageDraw, Image, ImageFilter , ImageOps

import torch
from diffusers import StableDiffusionImg2ImgPipeline
from torch.cuda.amp import autocast

torch.backends.cudnn.benchmark = True


c:\Users\A1234\Documents\projets\FONT_GENERATOR\fontsd_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\A1234\Documents\projets\FONT_GENERATOR\fontsd_env\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\A1234\Documents\projets\FONT_GENERATOR\fontsd_env\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\A1234\Documents\projets\FONT_GENERATOR\fontsd_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # TODO: remove that shit

def imshow(img):
    plt.tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)
    plt.imshow(img)
    plt.show()

In [3]:
#TODO: 
# - make class smaller
# - make this more efficient
# - remove all the print statements

class FontInit:
    
    def __init__(self, kwargs):
        
        for key, value in kwargs.items():
            setattr(self, key, value)
        
        self.W = 500
        self.H = 500
        
        try :
            if self.fontpath == '':
                self.fontpath = 'fonts/arial.ttf'
        except AttributeError:
            self.fontpath = 'fonts/arial.ttf'
        
    
    def get_init_image(self):
        """ Create the initial image with the text, color, noise, init image and contours """
        
        # print("inside get init image")
        #compute and process noise image
        noise_img = self.get_noise()
        # print("init image: noise image computed")
        
        # process init_photo if there is one
        if self.init_image is not None:
            self.init_image = Image.fromarray(self.init_image)
            self.init_image = self.init_image.resize((int(self.W), int(self.H))).convert('RGBA')
            # self.print('init image size', self.init_image.size)
        
        # blend noise and init image
        if self.init_image is not None:
            init_photo_noisy = Image.blend(self.init_image, noise_img, float(1-self.init_image_strength))
        else:
            init_photo_noisy = noise_img
        init_photo_noisy = Image.fromarray(np.array(init_photo_noisy)).convert("RGBA")
        # print("blended noise and init image")
        
        # compute font mask image
        text_mask = self.font_mask()
        # print("computed font mask")
        
        # compute contour of the text 
        contour = text_mask.filter(ImageFilter.CONTOUR()).convert('L')
        contour = np.array(contour)
        contour[:,0] = 255
        contour[0,:] = 255
        contour[-1,:] = 255
        contour[:,-1] = 255
        contour = Image.fromarray(contour)
        # print("computed contour")
        # thicker contour, blur and color 
        contour = contour.filter(ImageFilter.GaussianBlur(radius= self.contour_width))
        contour = contour.point(lambda x: 0 if x > 240 else 255, '1').convert('L')
        contour = contour.filter(ImageFilter.GaussianBlur(radius= self.contour_blur))
        contour = ImageOps.colorize(contour, white = self.contour_color, black ="black")
        # print(" thicker contour, blur and color")

        # put everything together
        #add the contour mask to the text mask
        text_mask = text_mask.filter(ImageFilter.GaussianBlur(radius= 5)).convert("L") # blur the text mask
        masked = Image.new("RGBA", init_photo_noisy.size, (0,0,0,0)) # create a transparent image
        masked = Image.composite(init_photo_noisy, contour, text_mask) # add the contour
        # masked = Image.composite(init_photo_noisy, masked_with_contour, text_mask) # add the text mask
        # print("end=put everything together")
        
        return np.array(masked)
    
    #---------------------------------------------------------------------------------------------------
    def get_optimal_fontsize(self):
        """ Get the optimal font size for the text to fit in the image """

        fontsize =1
        font = ImageFont.truetype(self.fontpath, fontsize)

        left, top, right, bottom = font.getbbox(self.txt)
        while abs(right-left) < self.image_fraction*self.W and abs(top-bottom) < self.image_fraction*self.H:
            # iterate until the text size is just larger than the criteria
            fontsize += 1
            font = ImageFont.truetype(self.fontpath, fontsize)
            left, top, right, bottom = font.getbbox(self.txt)

        # make sure it is not too large
        fontsize -= 1
        font = ImageFont.truetype(self.fontpath, fontsize)

        return font

    def font_mask(self): #was font_mask
        """ Create a black image with white text in the center """

        img = np.zeros((self.W, self.H), dtype = "uint8")
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        font = self.get_optimal_fontsize()
        draw.text((self.W/2, self.H/2), str(self.txt), font=font,fill='white', anchor='mm')
        
        return img
    #---------------------------------------------------------------------------------------------------
    def gaussian_noise_colored(self):
        """ returns a gaussian noise image with a specific color (multicolor noise) """
        noise = np.zeros((self.W, self.H, 3), np.uint8)
        cv2.randn(noise, np.zeros(3), np.ones(3)*255)
        color = np.array(Image.new('RGB', (500, 500), self.noise_color))
        return noise + color
    
    def gaussian_noise_colored_plain(self):
        """ returns a gaussian noise image with a specific color (BW noise) """
        print('inside gaussian_noise_colored_plain, self.noise_color=', self.noise_color)
        noise = np.zeros((self.W, self.H), np.uint8)
        noise = cv2.randn(noise, self.mean, self.var)
        noise = np.array([noise, noise, noise]).transpose(1,2,0)
        color = np.array(Image.new('RGB', (500, 500), self.noise_color))
        return noise + color
    
    def get_noise(self):
        """ returns a gaussian noise image """
        print('inside get_noise, self.noise_color=', self.noise_color)
        if self.multicolor_noise:
            noise_img = self.gaussian_noise_colored()
        else:
            noise_img = self.gaussian_noise_colored_plain()
        noise_img =  Image.fromarray(noise_img).convert('RGBA')
        return noise_img
    #---------------------------------------------------------------------------------------------------
        

In [4]:
import gradio as gr


# functions used for the first demo
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def fn(txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var):
    """
    returns the init font image
    args is a dict storing the parameters
    """
    #make a list of all the parameters
    args = [txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var]
    args_string = ['txt','fontpath', 'image_fraction', 'init_image', 'noise_color', 'init_image_strength', 'contour_bool', 'contour_color', 'contour_width', 'contour_blur', 'multicolor_noise', 'mean', 'var']
    argsdict = dict(zip(args_string, args))
    print('argsdict=', argsdict)
    initobject = FontInit(argsdict) # should not be an init but an update instead
    
    return initobject.get_init_image()

def compute_all_letters(txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var):
    """ 
    compute and save all pre-letters in a folder
    """
    letter_list = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789qwertyuiopasdfghjklzxcvbnm!,.:'
    #todo: possibility to use a custom set of letters
    args = [txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var]
    args_string = ['txt', 'fontpath', 'image_fraction', 'init_image', 'noise_color', 'init_image_strength', 'contour_bool', 'contour_color', 'contour_width', 'contour_blur', 'multicolor_noise', 'mean', 'var']
    argsdict = dict(zip(args_string, args))
    for letter in letter_list:
        argsdict['txt'] = letter
        initobject = FontInit(argsdict)
        img = initobject.get_init_image()
        img = Image.fromarray(img)
        # if letter exists in the directory, find another name
        if os.path.exists('init_images/'+letter+'.png'):
            letter = letter+'_bis'
        img.save('init_images/'+letter+'.png')
        print('saved letter ', letter)
    
    return None
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    

with gr.Blocks() as demo:
    # -----------------------------------------------------------------------
    gr.Markdown("FONT DIFFUSION")
    gr.Markdown("# 1 - Create your base letters")
    gr.Markdown("These will be used as input images in Stable Diffusion Img2Img pipeline...")
    
    with gr.Row():
        with gr.Box():
            
            # letter
            with gr.Box():
                with gr.Row():
                    txt = gr.Textbox(label="Letter") # txt
                    image_fraction = gr.Slider(0.2, 1, label='adjust size', value=0.8) # image_fraction
                    #get a fontlist from all the ttfs inside the font folder
                    fontlist = ['fonts/'+f for f in os.listdir('fonts') if f.lower().endswith('.ttf')]
                    print('fontlist', fontlist)
                    fontpath = gr.Dropdown(fontlist, label='base font')#fontpath choice
            with gr.Box():
                with gr.Row():
                    init_image = gr.Image(label='init_image') # init_image
                    noise_color = gr.ColorPicker(label= 'color') # noise_color
                    init_image_strength =  gr.Slider(0., 1., label='i_strength', value = 0.5) # init_image_strength
            
            #contour
            with gr.Box():
                with gr.Column():
                    # n = gr.Checkbox(label='contour')
                    contour_bool = gr.Checkbox(label='contour') # contour_bool
                    contour_color = gr.ColorPicker(label= 'contour color') # contour_color
                    contour_width = gr.Slider(1, 10, label='contour width') # contour_width
                    contour_blur = gr.Slider(0, 5, label='contour blur') # contour_blur
            
            # noise
            with gr.Box():
                with gr.Column():
                    multicolor_noise = gr.Checkbox(label='multicolor_noise') # multicolor_noise
                    mean = gr.Slider(0, 255, label='noise_mean', value=100) # mean
                    var = gr.Slider(0, 100, label='noise_var', value=20)# var
            
            #button to compute one letter and display it
            button = gr.Button("compute !")
            
        output = gr.components.Image()
        out = None
        
     
    args = [txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var]
    for a in args:
        a.change(fn, args, output)
        
    # handle alphabet button
    # button_alphabet.click(compute_all_letters, args, out)
    
    # initobject = FontInit(**args)
    # button.click(fn, args, output)
    
    # TO DO : MAKE A REACTIVE INTERFACE INSTEAD
   
    # -----------------------------------------------------------------------
    gr.Markdown("# 2 - Diffuse the rest")
    # Note: When using Stable Diffusion, black images will be returned if the given prompt is detected as problematic. For composing GLIDE model, we recommend using the Colab demo in our Project Page.
    
    #TODO:
    # make a local and a hugging face version
    # diffuser = gr.Interface.load("https://huggingface.co/spaces/fffiloni/stable-diffusion-img2img")
    
    # local version
    # make sure you're logged in with `huggingface-cli login`
    
    device = "cuda"
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True
    ).to(device)
    pipe.enable_attention_slicing()
    print('done loading pipeline :)')
    
    # to numpy array
    def compute_diffused_letter(prompt, init_image, strength):
        print('diffusing...')
        # init_image = np.array(output)
        print('init_image:', init_image)
        init_image = Image.fromarray(init_image).convert('RGB')
        with autocast(True):
            output_2 = pipe(prompt=prompt, init_image=init_image, strength=strength, guidance_scale=7.5).images[0]
        output_2.save("output.png")
        return output_2

    prompt = gr.Textbox(label="prompt")
    strength = gr.Slider(0., 1., label='strength', value=0.5)
    button_diffused = gr.Button("diffuse !")
    args2 = [prompt, output, strength]
    output_2 = gr.components.Image()
    button_diffused.click(compute_diffused_letter, args2, output_2)

    
    # -----------------------------------------------------------------------
    gr.Markdown("# 3 - Remove the background")
    
    
    # remove_bg = gr.Interface.load("spaces/eugenesiow/remove-bg")
    
    # -----------------------------------------------------------------------
    gr.Markdown("# 4 - Batch generate the letters you want")
    
    
    # todo: put this somewhere else...
    def diffuse_all_letters(txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var, custom_letters, prompt, strength):
        """" diffuses all the letters in the tmp folder and saves them in the output folder"""

        letter_list = custom_letters
        print('letter_list: ', letter_list)
        
        if letter_list is None:
            letter_list = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        elif letter_list =='':
            print('Please enter a list of letters ')
        # else:
        #     #remove the spaces
        #     letter_list = letter_list.replace(' ', '')
        #todo: possibility to use a custom set of letters
        args1 = [txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var]
        args_string = ['txt', 'fontpath', 'image_fraction', 'init_image', 'noise_color', 'init_image_strength', 'contour_bool', 'contour_color', 'contour_width', 'contour_blur', 'multicolor_noise', 'mean', 'var']
        argsdict = dict(zip(args_string, args1))
        for letter in letter_list:
            argsdict['txt'] = letter
            initobject = FontInit(argsdict)
            img = initobject.get_init_image()
            # img = Image.fromarray(img)
            
            strength = 0.5
            # init_image = Image.open(letter)
            init_image = img
            out_image = compute_diffused_letter(prompt, init_image, strength)
            out_image.save(f'output/{letter}.png')
            print('diffused letter:', letter)

            
    # TODO: COMPUTE THEM IN SOME TMP FOLDER
    custom_letters = gr.Textbox(label="custom letters")
    compute_letters = gr.Button("precompute letters")
    compute_alphabet = gr.Button("precompute alphabet")
    
    diffuse_letters = gr.Button("diffuse letters")
    diffuse_alphabet = gr.Button("diffuse alphabet")
    
    # out_progresion  = gr.components.Image()
    
    # compute_letters.click(compute_all_letters, args, out)
    compute_alphabet.click(compute_all_letters, args, out)
    args3 = [txt, fontpath, image_fraction, init_image, noise_color, init_image_strength, contour_bool, contour_color, contour_width, contour_blur, multicolor_noise, mean, var, custom_letters, prompt, strength]
    diffuse_letters.click(diffuse_all_letters, args3, out)
    diffuse_alphabet.click(diffuse_all_letters, args3, out)
    
    gr.Markdown("Antonin Wattel")
    gr.Markdown("September 2022")
    

demo.launch(debug=True)

fontlist ['fonts/1610_Cancellaresca_lim.TTF', 'fonts/Altera.ttf', 'fonts/ARABOLIC.TTF', 'fonts/Aramis Italic.ttf', 'fonts/bleedingwatercolor.ttf', 'fonts/Earltik.ttf', 'fonts/paddr___.ttf', 'fonts/ProcessingSansPro-Regular.ttf', 'fonts/ProcessingSansPro-Semibold.ttf', 'fonts/SourceCodePro-Bold.ttf', 'fonts/SourceCodePro-Regular.ttf', 'fonts/warriot.ttf', 'fonts/WinterthurCondensedOblique.ttf']


Fetching 16 files: 100%|██████████| 16/16 [00:00<00:00, 2666.44it/s]


done loading pipeline :)
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


argsdict= {'txt': 'A', 'fontpath': '', 'image_fraction': 0.8, 'init_image': None, 'noise_color': '#000000', 'init_image_strength': 0.5, 'contour_bool': False, 'contour_color': '#000000', 'contour_width': 1, 'contour_blur': 0, 'multicolor_noise': False, 'mean': 100, 'var': 20}
inside get_noise, self.noise_color= #000000
inside gaussian_noise_colored_plain, self.noise_color= #000000
argsdict= {'txt': 'A', 'fontpath': 'fonts/ARABOLIC.TTF', 'image_fraction': 0.8, 'init_image': None, 'noise_color': '#000000', 'init_image_strength': 0.5, 'contour_bool': False, 'contour_color': '#000000', 'contour_width': 1, 'contour_blur': 0, 'multicolor_noise': False, 'mean': 100, 'var': 20}
inside get_noise, self.noise_color= #000000
inside gaussian_noise_colored_plain, self.noise_color= #000000
argsdict= {'txt': 'A', 'fontpath': 'fonts/ProcessingSansPro-Semibold.ttf', 'image_fraction': 0.8, 'init_image': None, 'noise_color': '#000000', 'init_image_strength': 0.5, 'contour_bool': False, 'contour_color': '#

100%|██████████| 33/33 [00:15<00:00,  2.12it/s]


argsdict= {'txt': 'A', 'fontpath': 'fonts/SourceCodePro-Bold.ttf', 'image_fraction': 0.8, 'init_image': array([[[230, 224, 224],
        [229, 223, 223],
        [229, 223, 223],
        ...,
        [ 82,  61,  32],
        [ 83,  53,  27],
        [ 99,  67,  42]],

       [[232, 226, 226],
        [231, 225, 225],
        [231, 225, 225],
        ...,
        [ 87,  66,  37],
        [ 90,  64,  37],
        [101,  74,  45]],

       [[231, 225, 225],
        [230, 224, 224],
        [230, 224, 224],
        ...,
        [ 88,  71,  41],
        [ 97,  76,  45],
        [ 95,  75,  42]],

       ...,

       [[193, 155,  90],
        [194, 155,  90],
        [196, 147,  89],
        ...,
        [171, 127,  82],
        [187, 139,  90],
        [193, 143,  90]],

       [[202, 165,  97],
        [203, 164,  97],
        [199, 150,  91],
        ...,
        [172, 125,  81],
        [181, 131,  82],
        [190, 138,  88]],

       [[202, 165,  95],
        [207, 168, 101],
        

100%|██████████| 38/38 [00:15<00:00,  2.40it/s]


argsdict= {'txt': 'A', 'fontpath': 'fonts/warriot.ttf', 'image_fraction': 0.8, 'init_image': array([[[230, 224, 224],
        [229, 223, 223],
        [229, 223, 223],
        ...,
        [ 82,  61,  32],
        [ 83,  53,  27],
        [ 99,  67,  42]],

       [[232, 226, 226],
        [231, 225, 225],
        [231, 225, 225],
        ...,
        [ 87,  66,  37],
        [ 90,  64,  37],
        [101,  74,  45]],

       [[231, 225, 225],
        [230, 224, 224],
        [230, 224, 224],
        ...,
        [ 88,  71,  41],
        [ 97,  76,  45],
        [ 95,  75,  42]],

       ...,

       [[193, 155,  90],
        [194, 155,  90],
        [196, 147,  89],
        ...,
        [171, 127,  82],
        [187, 139,  90],
        [193, 143,  90]],

       [[202, 165,  97],
        [203, 164,  97],
        [199, 150,  91],
        ...,
        [172, 125,  81],
        [181, 131,  82],
        [190, 138,  88]],

       [[202, 165,  95],
        [207, 168, 101],
        [204, 155, 

100%|██████████| 44/44 [00:18<00:00,  2.40it/s]
